# Webscraper to get Canadian postal code data 

In [3]:
#!conda install -c conda-forge folium=0.5.0 --yes
import pandas as pd
import folium
import requests 
import numpy as np

There exist multiple ways to obtain data from the web when API's are not directly available.  
I will use pandas awesome method read_html to get the table I want.

First, I read all tables available within the Wikipedia link and chose the first one (pandas provides a list of dataframes), headers included.  


In [4]:
dfs = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M", header=0)
#I find the first table by printing 
#for df in dfs:
#    print(df)
df = dfs[0]
df.rename(columns={'Postcode': 'PostalCode', 'Neighbourhood':'Neighborhood'}, inplace=True)
#Count how many postalcodes exist per Borough
df.groupby('Borough').count()['PostalCode']

Borough
Central Toronto     17
Downtown Toronto    37
East Toronto         7
East York            6
Etobicoke           45
Mississauga          1
North York          38
Not assigned        77
Queen's Park         1
Scarborough         37
West Toronto        13
York                 9
Name: PostalCode, dtype: int64

77 entries are not assigned and hence will be dropped

In [5]:
df = df[df['Borough']!='Not assigned']
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


Now Postalcodes that contain multiple neighborhoods will be summarized and all neighborhoods are concatinated with comma seperation

In [6]:
df_new = df.groupby('PostalCode')['Neighborhood'].agg([('Neighborhood', ', '.join)]).reset_index()

This code below would do the same job, I played around with it but found the above one easier to read

In [7]:
#Alternative code with same results
#df_new = (df['Neighborhood'].str.split(', ')
#                    .groupby(df['PostalCode'])
#                    .agg(lambda x: ', '.join(set(y for z in x for y in z)))
#                    .reset_index())
#df_new.shape

Since the previous operation dropped the Borough column it will be merged  from the previous data frame

In [8]:
df_new = pd.merge(df_new,df[['PostalCode','Borough']],on='PostalCode', how='left')
df_new = df_new.drop_duplicates()

M7A contains neighborhood with value 'not assigned'. It will be replaced by its Borough

In [9]:
df_new.loc[df_new['Neighborhood'] == 'Not assigned']

,PostalCode,Neighborhood,Borough
159,M7A,Not assigned,Queen's Park


Last, replace  all Neighborhoods that are not assigned with their respective borough

In [10]:
df_new['Neighborhood'].where(df_new['Neighborhood']!='Not assigned', other=df_new['Borough'], inplace=True)
#df_new['Neighborhood'] = df_new['Borough'].where(df_new['Neighborhood'] == 'Not assigned')
#if error , ''

In [11]:
df_new.loc[df_new['Neighborhood'] == "Queen's Park"]

,PostalCode,Neighborhood,Borough
159,M7A,Queen's Park,Queen's Park


In [12]:
df_new.shape

(103, 3)

Now I load the latitude and longitude data from a csv file and merge the dataframes according to their PostalCode

In [13]:
df_loc = pd.read_csv('https://cocl.us/Geospatial_data')
df_loc.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
df_loc.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
df_combined = pd.merge(df_new,df_loc, on='PostalCode')

In [15]:
df_combined.head(10)

,PostalCode,Neighborhood,Borough,Latitude,Longitude
0,M1B,"Rouge, Malvern",Scarborough,43.806686,-79.194353
1,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough,43.784535,-79.160497
2,M1E,"Guildwood, Morningside, West Hill",Scarborough,43.763573,-79.188711
3,M1G,Woburn,Scarborough,43.770992,-79.216917
4,M1H,Cedarbrae,Scarborough,43.773136,-79.239476
5,M1J,Scarborough Village,Scarborough,43.744734,-79.239476
6,M1K,"East Birchmount Park, Ionview, Kennedy Park",Scarborough,43.727929,-79.262029
7,M1L,"Clairlea, Golden Mile, Oakridge",Scarborough,43.711112,-79.284577
8,M1M,"Cliffcrest, Cliffside, Scarborough Village West",Scarborough,43.716316,-79.239476
9,M1N,"Birch Cliff, Cliffside West",Scarborough,43.692657,-79.264848


# Clustering the Data and graph it

Get the geographical data for Toronto

In [16]:
latitude = 43.6532
longitude = -79.3832

Graph Toronto with its Boroughs

In [17]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(df_combined['Latitude'], df_combined['Longitude'], df_combined['Borough'], df_combined['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [18]:
# The code was removed by Watson Studio for sharing.

I'm using the Foursquare API to obtain venue information about Toronto  
Here the venue information for each Neighborhood is obtained

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT = 30
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
toronto_venues = getNearbyVenues(names=df_combined['Neighborhood'],
                                   latitudes=df_combined['Latitude'],
                                   longitudes=df_combined['Longitude']
                                  )


Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

After creating the toronto_venues data frame I count the venues downloaded for each neighborhood

In [22]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa


In [23]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",30,30,30,30,30,30
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3,3,3,3,3,3
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",9,9,9,9,9,9
"Alderwood, Long Branch",10,10,10,10,10,10
"Bathurst Manor, Downsview North, Wilson Heights",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
Berczy Park,30,30,30,30,30,30


Now the dataframe needs to be one hot encoded in order to obtain numerical data rather than categorical strings

In [24]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

### Clustering using k=4 clusters

In [26]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

The 10 most common venues are now added to the dataframe

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Steakhouse,Café,Hotel,Asian Restaurant,Pizza Place,Food Court,Seafood Restaurant,Breakfast Spot,Coffee Shop,Lounge
1,Agincourt,Lounge,Skating Rink,Breakfast Spot,Sandwich Place,Women's Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Coffee Shop,Park,Playground,Curling Ice,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Beer Store,Fried Chicken Joint,Fast Food Restaurant,Coffee Shop,Pizza Place,Sandwich Place,Pharmacy,Airport Service,Deli / Bodega
4,"Alderwood, Long Branch",Pizza Place,Sandwich Place,Coffee Shop,Athletics & Sports,Skating Rink,Gym,Pharmacy,Pub,Pool,Women's Store


In [28]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_combined

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.dropna(inplace=True)
toronto_merged["Cluster Labels"] = toronto_merged["Cluster Labels"].astype(int)

toronto_merged.head() # check the last columns!

,PostalCode,Neighborhood,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,"Rouge, Malvern",Scarborough,43.806686,-79.194353,0,Fast Food Restaurant,Women's Store,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner
1,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough,43.784535,-79.160497,0,Bar,Women's Store,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store
2,M1E,"Guildwood, Morningside, West Hill",Scarborough,43.763573,-79.188711,0,Electronics Store,Breakfast Spot,Rental Car Location,Intersection,Mexican Restaurant,Medical Center,Pizza Place,Spa,Discount Store,Diner
3,M1G,Woburn,Scarborough,43.770992,-79.216917,0,Coffee Shop,Korean Restaurant,Insurance Office,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store
4,M1H,Cedarbrae,Scarborough,43.773136,-79.239476,0,Caribbean Restaurant,Fried Chicken Joint,Hakka Restaurant,Thai Restaurant,Bank,Athletics & Sports,Bakery,Discount Store,Dessert Shop,Dim Sum Restaurant


In [29]:
# create map
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Cluster 1

By far the biggest cluster, including the most common and probably most visited venues. Restaurants, shops, Banks and stores. They seem evenly represented.

In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Rouge, Malvern",0,Fast Food Restaurant,Women's Store,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner
1,"Highland Creek, Rouge Hill, Port Union",0,Bar,Women's Store,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store
2,"Guildwood, Morningside, West Hill",0,Electronics Store,Breakfast Spot,Rental Car Location,Intersection,Mexican Restaurant,Medical Center,Pizza Place,Spa,Discount Store,Diner
3,Woburn,0,Coffee Shop,Korean Restaurant,Insurance Office,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store
4,Cedarbrae,0,Caribbean Restaurant,Fried Chicken Joint,Hakka Restaurant,Thai Restaurant,Bank,Athletics & Sports,Bakery,Discount Store,Dessert Shop,Dim Sum Restaurant
5,Scarborough Village,0,Playground,Convenience Store,Women's Store,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store
6,"East Birchmount Park, Ionview, Kennedy Park",0,Coffee Shop,Department Store,Discount Store,Bus Station,College Stadium,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore
7,"Clairlea, Golden Mile, Oakridge",0,Bus Line,Bakery,Soccer Field,Fast Food Restaurant,Metro Station,Intersection,Park,Concert Hall,Dessert Shop,Eastern European Restaurant
9,"Birch Cliff, Cliffside West",0,College Stadium,General Entertainment,Skating Rink,Café,Women's Store,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run
10,"Dorset Park, Scarborough Town Centre, Wexford ...",0,Indian Restaurant,Vietnamese Restaurant,Latin American Restaurant,Chinese Restaurant,Pet Store,College Gym,Dance Studio,Eastern European Restaurant,Coffee Shop,Dumpling Restaurant


## Custer 2

Charactarized by having Baseball Fields as the first most common venue.

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,"Humber Bay, King's Mill Park, Kingsway Park So...",1,Baseball Field,Women's Store,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store
97,"Emery, Humberlea",1,Baseball Field,Construction & Landscaping,Women's Store,Deli / Bodega,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run


## Cluster 3

Here Parks dominate the most common venues. It is remarkable how the unsupervised KMeans method identified them to be grouped together.

In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,"Agincourt North, L'Amoreaux East, Milliken, St...",2,Coffee Shop,Park,Playground,Curling Ice,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner
20,"Silver Hills, York Mills",2,Park,Cafeteria,Women's Store,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store
23,York Mills West,2,Park,Electronics Store,Bank,Women's Store,Deli / Bodega,Empanada Restaurant,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run
30,"CFB Toronto, Downsview East",2,Park,Snack Place,Airport,Women's Store,Curling Ice,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store
40,East Toronto,2,Park,Metro Station,Convenience Store,Women's Store,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store
50,Rosedale,2,Park,Playground,Trail,Cuban Restaurant,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner
74,Caledonia-Fairbanks,2,Park,Women's Store,Fast Food Restaurant,Market,Pharmacy,Airport Food Court,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
90,"The Kingsway, Montgomery Road, Old Mill North",2,River,Park,Cuban Restaurant,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner,Dim Sum Restaurant
98,Weston,2,Park,Women's Store,Curling Ice,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner


## Cluster 4

This cluster consists of only one element. It is a borough where motels dominate the list of most common venues.

In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,"Cliffcrest, Cliffside, Scarborough Village West",3,Motel,American Restaurant,Dance Studio,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store
